In [1]:
import pandas as pd
import ast
import os
from pprint import pprint
from auxiliar_func import *

In [ ]:
# show the best params for all the files in the /results folder
for file in os.listdir('./results'):
    if file.endswith('.csv'):
        print('='*20+file+'='*20)
        prep_p, mod_p = get_best_params('./results/'+file)
        prep_p = pd.DataFrame.from_dict(prep_p, orient='index', columns=['value'])
        mod_p = pd.DataFrame.from_dict(mod_p, orient='index', columns=['value'])
        print('Preprocessing best params:')
        print(prep_p.head())
        print('Model best params:')
        print(mod_p.head())

In [ ]:
df = pd.read_csv('./results/results_lda.csv')
for obj_col in ['prep_param', 'model_param']:
    df[obj_col] = df[obj_col].apply(lambda x: ast.literal_eval(x))
df.sort_values(by=['f1_macro'], inplace=True, ascending=False)
df.drop_duplicates(subset=['f1_macro'], inplace=True)
df_exp = expand_dicts(df)
df_exp.head()

In [ ]:
df = pd.read_csv('results_log_regression.csv')
df.sort_values(by=['f1_macro'], inplace=True, ascending=False)
df.head()

In [5]:
df = read_results('./results/results_rf.csv')
df = expand_dicts(df)
df['n_estimators'].unique()

array([ 75, 100,  50], dtype=int64)

In [ ]:
df = pd.read_csv('results_svm.csv')
df.sort_values(by=['f1_macro'], inplace=True, ascending=False)
df.head()

In [ ]:
df = pd.read_csv('results_catboost.csv')
df.sort_values(by=['f1_macro'], inplace=True, ascending=False)
df.head()